<h1><p style="color: green">Conclusion</p></h1>

In [1]:
from prettytable import PrettyTable

In [19]:
ptable1 = PrettyTable()
ptable1.title = " Model Comparision "
ptable1.field_names = ['Model Name','Hyperparameter Tunning', 'Accuracy']

print("\n\n ********** Machine Learning Model Comparision ************")
ptable1.add_row(["Logistic Regression","Done","95.83%"])
ptable1.add_row(["Linear SVC  ","Done","96.47%"])
ptable1.add_row(["rbf SVM classifier","Done","96.27%"])
ptable1.add_row(["DecisionTree","Done","86.46%"])
ptable1.add_row(["Random Forest","Done","92.6%"])

print(ptable1)
# *****************************************************************

ptable2 = PrettyTable()
ptable2.title = " Model Comparision "
ptable2.field_names = ['Model Name','Hyperparameter Tunning', 'categorical_crossentropy', 'Accuracy']

print("\n\n ********************************* Deep Learning LSTM Model Comparision ***********************************")
ptable2.add_row(["LSTM With 1_Layer(neurons:32)","Done","0.47", "0.90"])
ptable2.add_row(["LSTM With 2_Layer(neurons:48, neurons:32)","Done","0.39", "0.90"])
ptable2.add_row(["LSTM With 2_Layer(neurons:64, neurons:48)","Done","0.27", "0.91"])

print(ptable2)



 ********** Machine Learning Model Comparision ************
+---------------------+------------------------+----------+
|      Model Name     | Hyperparameter Tunning | Accuracy |
+---------------------+------------------------+----------+
| Logistic Regression |          Done          |  95.83%  |
|     Linear SVC      |          Done          |  96.47%  |
|  rbf SVM classifier |          Done          |  96.27%  |
|     DecisionTree    |          Done          |  86.46%  |
|    Random Forest    |          Done          |  92.6%   |
+---------------------+------------------------+----------+


 ********************************* Deep Learning LSTM Model Comparision ***********************************
+-------------------------------------------+------------------------+--------------------------+----------+
|                 Model Name                | Hyperparameter Tunning | categorical_crossentropy | Accuracy |
+-------------------------------------------+------------------------+

1. ___Machine Learning Model Comparision___ :  We can choose ___Linear SVC___ or ___rbf SVM classifier___ or ___Logistic Regression___ as our best model while applying ML Classical Model.

2. ___Deep Learning LSTM Model Comparision___ : When we talking about LSTM Model, here with LSTM we are using simple RAW data(in ML model we are using Single engineered data made by an expert), but we can see the result without any FE data, LSTM perform very-very well and got highest 91% accuracy with 2_layer LSTM with hyperparameter Tunning and also we can clearly when we are increasing LSTM layer and Hyperparameter Tunning the cross-entropy value is decreasing and Accuracy is increasing.

In [ ]:
import numpy as np

def baum_welch(observed_sequence, num_states, num_observations, num_iterations):
    # Initialization
    transition_matrix = np.random.rand(num_states, num_states)
    transition_matrix /= np.sum(transition_matrix, axis=1, keepdims=True)
    
    emission_matrix = np.random.rand(num_states, num_observations)
    emission_matrix /= np.sum(emission_matrix, axis=1, keepdims=True)
    
    initial_state_probs = np.random.rand(num_states)
    initial_state_probs /= np.sum(initial_state_probs)
    
    # Iterate through the Baum-Welch algorithm
    for iteration in range(num_iterations):
        # Forward pass
        forward_probs = forward(observed_sequence, transition_matrix, emission_matrix, initial_state_probs)
        
        # Backward pass
        backward_probs = backward(observed_sequence, transition_matrix, emission_matrix)
        
        # Expectation step
        expected_state_counts, expected_transition_counts, expected_observation_counts = \
            expectation(observed_sequence, forward_probs, backward_probs, transition_matrix, emission_matrix)
        
        # Maximization step
        transition_matrix, emission_matrix, initial_state_probs = \
            maximization(expected_state_counts, expected_transition_counts, expected_observation_counts)
    
    return transition_matrix, emission_matrix, initial_state_probs

def forward(observed_sequence, transition_matrix, emission_matrix, initial_state_probs):
    num_states = transition_matrix.shape[0]
    num_observations = len(observed_sequence)
    
    forward_probs = np.zeros((num_states, num_observations))
    forward_probs[:, 0] = initial_state_probs * emission_matrix[:, observed_sequence[0]]
    
    for t in range(1, num_observations):
        for j in range(num_states):
            forward_probs[j, t] = np.sum(forward_probs[:, t-1] * transition_matrix[:, j]) * emission_matrix[j, observed_sequence[t]]
    
    return forward_probs

def backward(observed_sequence, transition_matrix, emission_matrix):
    num_states = transition_matrix.shape[0]
    num_observations = len(observed_sequence)
    
    backward_probs = np.zeros((num_states, num_observations))
    backward_probs[:, -1] = 1.0
    
    for t in range(num_observations-2, -1, -1):
        for i in range(num_states):
            backward_probs[i, t] = np.sum(transition_matrix[i, :] * emission_matrix[:, observed_sequence[t+1]] * backward_probs[:, t+1])
    
    return backward_probs

def expectation(observed_sequence, forward_probs, backward_probs, transition_matrix, emission_matrix):
    num_states = transition_matrix.shape[0]
    num_observations = len(observed_sequence)
    
    expected_state_counts = forward_probs * backward_probs
    expected_state_counts /= np.sum(expected_state_counts, axis=0, keepdims=True)
    
    expected_transition_counts = np.zeros((num_states, num_states, num_observations-1))
    expected_observation_counts = np.zeros((num_states, num_observations))
    
    for t in range(num_observations-1):
        expected_transition_counts[:, :, t] = \
            (forward_probs[:, t][:, np.newaxis] * transition_matrix * emission_matrix[:, observed_sequence[t+1]] * backward_probs[:, t+1]).T
        expected_transition_counts[:, :, t] /= np.sum(expected_transition_counts[:, :, t])
        
        expected_observation_counts[:, t] = expected_state_counts[:, t] * (emission_matrix[:, observed_sequence[t]] / np.sum(emission_matrix[:, observed_sequence[t]]))
    
def maximization(expected_state_counts, expected_transition_counts, expected_observation_counts):
    num_states = expected_state_counts.shape[0]
    num_observations = expected_observation_counts.shape[1]
    
    # Update transition matrix
    updated_transition_matrix = np.sum(expected_transition_counts, axis=2)
    updated_transition_matrix /= np.sum(updated_transition_matrix, axis=1, keepdims=True)
    
    # Update emission matrix
    updated_emission_matrix = np.zeros_like(expected_observation_counts)
    
    for i in range(num_states):
        for k in range(num_observations):
            updated_emission_matrix[i, k] = np.sum(expected_observation_counts[i, :][observed_sequence == k])
    
    updated_emission_matrix /= np.sum(updated_emission_matrix, axis=1, keepdims=True)
    
    # Update initial state probabilities
    updated_initial_state_probs = expected_state_counts[:, 0]
    updated_initial_state_probs /= np.sum(updated_initial_state_probs)
    
    return updated_transition_matrix, updated_emission_matrix, updated_initial_state_probs


In [ ]:
import numpy as np


#generating initial probabilities

#transition probabilities
transition = np.array([[0.8,0.1],
                       [0.1,0.8]])
#Emission probabilities
emission = np.array([[0.1,0.2,0.7],
                     [0.7,0.2,0.1]])

#defining states and sequence symbols
states = ['H','C']
states_dic = {'H':0, 'C':1}
sequence_syms = {'1':0,'2':1,'3':2}
sequence = ['1','2','3']

#test sequence
test_sequence = '331122313'
test_sequence = [x for x in test_sequence]

#probabilities of going to end state
end_probs = [0.1, 0.1]
#probabilities of going from start state
start_probs = [0.5, 0.5]


#function to find forward probabilities
def forward_probs():
    # node values stored during forward algorithm
    node_values_fwd = np.zeros((len(states), len(test_sequence)))

    for i, sequence_val in enumerate(test_sequence):
        for j in range(len(states)):
            # if first sequence value then do this
            if (i == 0):
                node_values_fwd[j, i] = start_probs[j] * emission[j, sequence_syms[sequence_val]]
            # else perform this
            else:
                values = [node_values_fwd[k, i - 1] * emission[j, sequence_syms[sequence_val]] * transition[k, j] for k in
                          range(len(states))]
                node_values_fwd[j, i] = sum(values)

    #end state value
    end_state = np.multiply(node_values_fwd[:, -1], end_probs)
    end_state_val = sum(end_state)
    return node_values_fwd, end_state_val



#function to find backward probabilities
def backward_probs():
    # node values stored during forward algorithm
    node_values_bwd = np.zeros((len(states), len(test_sequence)))

    #for i, sequence_val in enumerate(test_sequence):
    for i in range(1,len(test_sequence)+1):
        for j in range(len(states)):
            # if first sequence value then do this
            if (-i == -1):
                node_values_bwd[j, -i] = end_probs[j]
            # else perform this
            else:
                values = [node_values_bwd[k, -i+1] * emission[k, sequence_syms[test_sequence[-i+1]]] * transition[j, k] for k in range(len(states))]
                node_values_bwd[j, -i] = sum(values)

    #start state value
    start_state = [node_values_bwd[m,0] * emission[m, sequence_syms[test_sequence[0]]] for m in range(len(states))]
    start_state = np.multiply(start_state, start_probs)
    start_state_val = sum(start_state)
    return node_values_bwd, start_state_val


#function to find si probabilities
def si_probs(forward, backward, forward_val):

    si_probabilities = np.zeros((len(states), len(test_sequence)-1, len(states)))

    for i in range(len(test_sequence)-1):
        for j in range(len(states)):
            for k in range(len(states)):
                si_probabilities[j,i,k] = ( forward[j,i] * backward[k,i+1] * transition[j,k] * emission[k,sequence_syms[test_sequence[i+1]]] ) \
                                                    / forward_val
    return si_probabilities

#function to find gamma probabilities
def gamma_probs(forward, backward, forward_val):

    gamma_probabilities = np.zeros((len(states), len(test_sequence)))

    for i in range(len(test_sequence)):
        for j in range(len(states)):
            #gamma_probabilities[j,i] = ( forward[j,i] * backward[j,i] * emission[j,sequence_syms[test_sequence[i]]] ) / forward_val
            gamma_probabilities[j, i] = (forward[j, i] * backward[j, i]) / forward_val

    return gamma_probabilities



#performing iterations until convergence

for iteration in range(2000):

    print('\nIteration No: ', iteration + 1)
    # print('\nTransition:\n ', transition)
    # print('\nEmission: \n', emission)

    #Calling probability functions to calculate all probabilities
    fwd_probs, fwd_val = forward_probs()
    bwd_probs, bwd_val = backward_probs()
    si_probabilities = si_probs(fwd_probs, bwd_probs, fwd_val)
    gamma_probabilities = gamma_probs(fwd_probs, bwd_probs, fwd_val)

    # print('Forward Probs:')
    # print(np.matrix(fwd_probs))
    #
    # print('Backward Probs:')
    # print(np.matrix(bwd_probs))
    #
    # print('Si Probs:')
    # print(si_probabilities)
    #
    # print('Gamma Probs:')
    # print(np.matrix(gamma_probabilities))

    #caclculating 'a' and 'b' matrices
    a = np.zeros((len(states), len(states)))
    b = np.zeros((len(states), len(sequence_syms)))

    #'a' matrix
    for j in range(len(states)):
        for i in range(len(states)):
            for t in range(len(test_sequence)-1):
                a[j,i] = a[j,i] + si_probabilities[j,t,i]

            denomenator_a = [si_probabilities[j, t_x, i_x] for t_x in range(len(test_sequence) - 1) for i_x in range(len(states))]
            denomenator_a = sum(denomenator_a)

            if (denomenator_a == 0):
                a[j,i] = 0
            else:
                a[j,i] = a[j,i]/denomenator_a

    #'b' matrix
    for j in range(len(states)): #states
        for i in range(len(sequence)): #seq
            indices = [idx for idx, val in enumerate(test_sequence) if val == sequence[i]]
            numerator_b = sum( gamma_probabilities[j,indices] )
            denomenator_b = sum( gamma_probabilities[j,:] )

            if (denomenator_b == 0):
                b[j,i] = 0
            else:
                b[j, i] = numerator_b / denomenator_b


    print('\nMatrix a:\n')
    print(np.matrix(a.round(decimals=4)))
    print('\nMatrix b:\n')
    print(np.matrix(b.round(decimals=4)))

    transition = a
    emission = b

    new_fwd_temp, new_fwd_temp_val = forward_probs()
    print('New forward probability: ', new_fwd_temp_val)
    diff =  np.abs(fwd_val - new_fwd_temp_val)
    print('Difference in forward probability: ', diff)

    if (diff < 0.0000001):
        break
